###### curve fitting done on AWS linux machine ( pyjags and pymc not available on windows)..so get parameters of all fitted curves parameters (ic50, m)  stored in csv format from Aanchal_AWS/output_files folder

### Script to
1. Plot fitted curves on union of 2 datasets (using updated fitted curves with 5 dosage drugs and replicates summarized in gdcs1)
2. Exract features from the curves for classifying them using area and clustering in next script (th on Area, no more used)

In [1]:
__author__ = 'Aanchal'

In [2]:
shrinkage=''
#shrinkage='shrinkage_'

In [3]:
import pandas as pd
import FUNCTIONS as f
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [4]:
#old data without 5 dosage drugs
#datasets_list=['CCLE','GDSC']
#rawFileDir="../data/"

datasets_list=['CCLE','GDSC1replicatesSummarized']
rawFileDir="../data/data_2019_12_05/"

sample_col_name='cell line name'
drug_col_name ='drug name'
dosage_col_name='Dosage (uM)' 
response_col_name='Response'

#directory containing m,ic50 of all fitte curves per pair (1 file per CL)
FittedCurves_dir='../../Aanchal_AWS/output_files/FittedCurves_commonPairs_replicatesSummarized/'

out_dir='../figures/FittedCurves_commonPairs_plots/'


In [5]:
[Intersection,  Intersection_names] = f.getCommonPairs(datasets_list, rawFileDir, sample_col_name, drug_col_name, dosage_col_name, response_col_name)

0
1


In [6]:
# FOR ALL COMMON PAIRS
indices=len(Intersection)
df=Intersection_names

In [7]:
# FOR 1 SPECIFIC PAIR
'''indices1=Intersection_names.index[Intersection_names['Standard cell line name']=='JVM-3']
indices2=Intersection_names.index[Intersection_names['Standard drug name']=='Paclitaxel']
#697,Paclitaxel

indices=Intersection_names.index[ indices1 & indices2]
df=Intersection_names.loc[indices]'''

"indices1=Intersection_names.index[Intersection_names['Standard cell line name']=='JVM-3']\nindices2=Intersection_names.index[Intersection_names['Standard drug name']=='Paclitaxel']\n#697,Paclitaxel\n\nindices=Intersection_names.index[ indices1 & indices2]\ndf=Intersection_names.loc[indices]"

In [8]:
# FOR 1 SPECIFIC DRUG/CL
'''drname='Selumetinib'
indices=Intersection_names.index[Intersection_names['Standard drug name']==drname1]
df=Intersection_names[Intersection_names['Standard drug name']==drname1]'''

#clname1='JVM-3'OCUM-1
'''clname1='TE-11'#'TE-11'
indices=Intersection_names.index[Intersection_names['Standard cell line name']==clname1]
df=Intersection_names[Intersection_names['Standard cell line name']==clname1]'''

"clname1='TE-11'#'TE-11'\nindices=Intersection_names.index[Intersection_names['Standard cell line name']==clname1]\ndf=Intersection_names[Intersection_names['Standard cell line name']==clname1]"

#### Generating HDI features

In [13]:
# dont use iloc, use iterrows .. DONE although no difference in runnig time
#vectorize, dont put loop .. DONE

hdi_a=[]
hdi_ab=[]
hdi_b=[]
hdi_bc=[]
hdi_c=[]
hdi_m=[]
hdi_ic50=[]

clname_l=[]
drname_l=[]

ar_fittedcurves_l=[]
ic50_l=[]
m_l=[]
mae_l=[]


width_l=[]
mindosage_l=[]
maxdosage_l=[]

cl_list = sorted(list(set(df[sample_col_name])))

i=0
for clname in cl_list:
    print(i, clname)
    i=i+1
    
    cl_df = df[df[sample_col_name] == clname]
    fin=os.path.join(FittedCurves_dir, '{:s}.csv'.format(clname +'_'+shrinkage+'fitted'))
    curveParas_df=pd.read_csv(fin)
    
    for ind, row in cl_df.iterrows():
        
        drname= row[drug_col_name]#drname=Intersection_names.loc[i,'Standard drug name']
        
        curveParas_df_drugSpecific=curveParas_df[curveParas_df['drug name']==drname]
        beta0=np.array(  curveParas_df_drugSpecific[['beta0']]  ) #hdi_ic50=getHDI(beta0)
        beta1=np.array( curveParas_df_drugSpecific[['beta1']]       ) #hdi_m=getHDI(beta1) 
        
        [log2_dosage, res_0to1, source_dataset]=f.get_pairInfo(Intersection, clname,drname, sample_col_name, drug_col_name, dosage_col_name, response_col_name)
        #label= [ 1+datasets_list.index(l) for l in source_dataset] #1+datasets_list.index[l]
       
        '''response_func=np.vectorize(f.getResponse)  
        
    
        [interval0, int_st0, int_end0] = f.getHDI( beta0, 0.99 )
        beta0mode = np.mean( [int_st0[0] , int_end0[0] ] ) #or int_st[0]+( int_end[0] -int_st[0] )/2 (mid value of 1%ile HDI)
        
        [interval1, int_st1, int_end1] = f.getHDI( beta1, 0.99 )
        beta1mode = np.mean( [int_st1[0] , int_end1[0] ] )

        
        mae_l.append( mean_absolute_error(res_0to1, response_func(log2_dosage,beta0mode,beta1mode)) )
        ic50_l.append(beta0mode)
        m_l.append(beta1mode)'''

        [mindosage, ab_dosage, middosage, bc_dosage, maxdosage]= f.get5dosagePoints(log2_dosage)
        
        #mindosage_l.append(mindosage)
        #maxdosage_l.append(maxdosage)
        #width_l.append(maxdosage-mindosage)
    
        xaxis=np.array( [mindosage, ab_dosage,middosage, bc_dosage ,maxdosage] ).transpose()
        [curve1, curve2]=f.get5DiscretePointsAtFittedCurves(mindosage, ab_dosage, middosage, bc_dosage, maxdosage,beta0,beta1)
        
        
        [ar_fittedcurves, dtw ]= f.getCurveSimilarities(xaxis , curve1, xaxis, curve2 ) #using older version of similaritymeasures in the form of code in FUNCTIONS file
        clname_l.append(clname)
        drname_l.append(drname)
        ar_fittedcurves_l.append(ar_fittedcurves)
        
    
        '''hdi_a.append( f.getHDI( response_func(mindosage,beta0,beta1), 0.05 )[0][0]  )
        hdi_ab.append( f.getHDI( response_func(ab_dosage,beta0,beta1), 0.05 )[0][0]  )
        hdi_b.append(f.getHDI( response_func(middosage,beta0,beta1), 0.05 )[0][0]  )
        hdi_bc.append(f.getHDI( response_func(bc_dosage,beta0,beta1), 0.05 )[0][0]  )
        hdi_c.append(f.getHDI( response_func(maxdosage,beta0,beta1), 0.05 )[0][0]  )
        hdi_m.append(f.getHDI( beta1, 0.05 )[0][0]  )
        hdi_ic50.append(f.getHDI( beta0, 0.05 )[0][0] )'''
        
        # plot  
        #f.plotFittedCurves(log2_dosage,res_0to1, beta0,beta1,  clname, drname,label, out_dir)
        

0 22RV1
1 42-MG-BA
2 5637
3 639-V
4 697
5 769-P
6 786-0
7 8-MG-BA
8 8305C_1
9 8505C
10 A172
11 A204
12 A2058
13 A253
14 A2780
15 A375
16 A549
17 A673
18 ACHN
19 ALL-SIL
20 AMO-1
21 AN3-CA
22 AU565
23 AsPC-1
24 BCPAP
25 BFTC-909
26 BL-41
27 BT-20
28 BT-474
29 BT-549
30 BxPC-3
31 C2BBe1
32 C32
33 C3A
34 CAL-12T
35 CAL-27
36 CAL-78
37 CAL-85-1
38 CAMA-1
39 CAS-1
40 CCK-81
41 CHL-1
42 CHP-212
43 CMK
44 COLO-205
45 COLO-320-HSR
46 COLO-678
47 COLO-679
48 COR-L105
49 COR-L23
50 Calu-3
51 Calu-6
52 Capan-2
53 DBTRG-05MG
54 DEL
55 DK-MG
56 DMS-114
57 DOHH-2
58 DOV13
59 DU-145
60 Daoy
61 Detroit562
62 EB2
63 EBC-1
64 EFM-19
65 EFO-21
66 EFO-27
67 EM-2
68 EN
69 ES-2
70 ESS-1
71 FADU
72 FU-OV-1
73 FU97
74 G-361
75 G-401
76 G-402
77 GAMG
78 GB-1
79 GCIY
80 GCT
81 GI-1
82 GMS-10
83 GRANTA-519
84 H4
85 HARA
86 HCC-15
87 HCC-44
88 HCC-78
89 HCC-827
90 HCC1187
91 HCC1395
92 HCC1569
93 HCC1806
94 HCC1954
95 HCC70
96 HCT-116
97 HCT-15
98 HD-MY-Z
99 HDQ-P1
100 HEL
101 HGC-27
102 HH
103 HLE
104 HOS
105 HP

In [23]:
'''f = pd.read_csv('../data_processed/FittedCurvesCommonPairs_features/Feature_Matrix_FLatPerDrugFitting_15features.csv')
f['Area_between_fitted_curves_5dosages']=ar_fittedcurves_l
f['IS']=f['Area_between_fitted_curves_5dosages']/f['width']
f.to_csv('../data_processed/FittedCurvesCommonPairs_features/Feature_Matrix_FLatPerDrugFitting_15features.csv', index=False)'''


In [ ]:
feature_df=pd.DataFrame()
feature_df['Standard cell line name']=clname_l
feature_df['Standard drug name']=drname_l
feature_df['HDI_mindosage']=hdi_a
feature_df['HDI_ab_dosage']=hdi_ab
feature_df['HDI_middosage']=hdi_b
feature_df['HDI_bc_dosage']=hdi_bc
feature_df['HDI_maxdosage']=hdi_c
feature_df['HDI_ic50']=hdi_ic50
feature_df['HDI_m']=hdi_m
feature_df['Area_between_fitted_curves_5dosages']=ar_fittedcurves_l
feature_df['m_mode']=m_l
feature_df['ic50_mode']=ic50_l
feature_df['mae_fitness']=mae_l
feature_df.sort_values(['Standard cell line name', 'Standard drug name'])

feature_df['width']=width_l
feature_df['log2(min_dosage_bothDatasets)']=mindosage_l
feature_df['log2(max_dosage_bothDatasets)']=maxdosage_l
feature_df['IS']=feature_df['Area_between_fitted_curves_5dosages']/feature_df['width'] #IS(inconsistency score) or AAA(Adjusted Approximayed area)
#feature_df.to_csv('../data_processed/FittedCurvesCommonPairs_features/Feature_Matrix_FLatPerDrugFitting_15features.csv', index=False)


In [ ]:
feature_df=feature_df.sort_values(['Standard cell line name', 'Standard drug name'])
feature_df